<a href="https://colab.research.google.com/github/SIDIBEMoussa/NLP-Trainings/blob/main/03_Exercice_LM_classification_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [88]:
import pathlib, os

In [89]:
path = "/content/datasets/lda_sports_politics_docs"

file_directory = pathlib.Path(path).exists()

if not file_directory:

    !git clone https://github.com/selva86/datasets.git



text_files = os.listdir(path)

In [90]:
text_files

['shinzo abe.txt',
 'pizza.txt',
 'lee quan yew.txt',
 'queen elizabeth.txt',
 'cricket.txt',
 'idli.txt',
 'barack obama.txt',
 'baseball.txt',
 'badminton.txt',
 'noodles.txt',
 'tipu sultan.txt',
 'table tennis.txt',
 'pasta.txt',
 'narendra modi.txt',
 'dosa.txt']

In [91]:
articles=[]

for filename in text_files:

    article = pathlib.Path(path+'/'+filename).read_text()

    articles.append(article)

In [92]:
articles[0]

"Shinzō Abe (Abe Shinzō, IPA: born 21 September 1954) is the current Prime Minister of Japan, re-elected to the position in December 2012. Abe is also the President of the Liberal Democratic Party (LDP).\nAbe served for a year as Prime Minister, from 2006 to 2007. Hailing from a politically prominent family, at age 52, Abe became Japan's youngest post-war prime minister, and the first to be born after World War II, when he was elected by a special session of the National Diet in September 2006. Abe resigned on 12 September 2007, for health reasons. Abe was replaced by Yasuo Fukuda, beginning a string of five Prime Ministers, none of whom retained office for more than sixteen months, before Abe staged a political comeback.\nOn 26 September 2012, Abe defeated former Minister of Defense Shigeru Ishiba, in a run-off vote, to win the LDP Presidential Election. Following the LDP's landslide victory in the 2012 general election, Abe became the Prime Minister again. Abe is the first former Pri

In [93]:
from pprint import pprint # for printing objects nicely

from gensim import corpora, models
from gensim.utils import simple_preprocess

# from gensim.parsing.preprocessing import STOPWORDS 
# Instead of the gensim english stopwords, we use nltk's German stopwords
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

import numpy as np

from random import choice

np.random.seed(1234)

stemmer = SnowballStemmer('english')
german_stop_words = set(stopwords.words('english'))


def lemmatize_stemming(text):
  return stemmer.stem(text)

def preprocess(text):
  result = [lemmatize_stemming(token) 
            for token in simple_preprocess(text)  
            if token not in german_stop_words and len(token) > 3]
  return result

In [94]:
print('original document: ')
article = choice(articles)
print(article)

# This time, we don't care about punctuations as tokens (Can you think why?):
print('original document, broken into words: ')
words = [word for word in article.split(' ')]
print(words)
print("Vocabulary size of the original article:", len(set(words)))

# now let's see what happens when we pass the article into our preprocessing 
# method:
print('\n\n tokenized and lemmatized document: ')
preprocessed_article = preprocess(article)
print(preprocessed_article)
print("Vocabulary size after preprocessing:", len(set(preprocessed_article)))

original document: 
Elizabeth II (Elizabeth Alexandra Mary; born 21 April 1926) is, and has been since her accession in 1952, Queen of theUnited Kingdom, Canada, Australia, and New Zealand, and Head of the Commonwealth. She is also Queen of 12 countries that have become independent since her accession: Jamaica, Barbados, the Bahamas, Grenada, Papua New Guinea,Solomon Islands, Tuvalu, Saint Lucia, Saint Vincent and the Grenadines, Belize, Antigua and Barbuda, and Saint Kitts and Nevis.
Elizabeth was born in London to the Duke and Duchess of York, later King George VI and Queen Elizabeth, and was the elder of their two daughters. She was educated privately at home. Her father acceded to the throne on the abdication of his brother Edward VIII in 1936, from which time she was the heir presumptive. She began to undertake public duties during theSecond World War, serving in the Auxiliary Territorial Service. In 1947, she married Philip, Duke of Edinburgh, with whom she has four children: Cha

In [95]:
processed_docs = list(map(preprocess, articles))
processed_docs[:2]

[['shinzō',
  'shinzō',
  'born',
  'septemb',
  'current',
  'prime',
  'minist',
  'japan',
  'elect',
  'posit',
  'decemb',
  'also',
  'presid',
  'liber',
  'democrat',
  'parti',
  'serv',
  'year',
  'prime',
  'minist',
  'hail',
  'polit',
  'promin',
  'famili',
  'becam',
  'japan',
  'youngest',
  'post',
  'prime',
  'minist',
  'first',
  'born',
  'world',
  'elect',
  'special',
  'session',
  'nation',
  'diet',
  'septemb',
  'resign',
  'septemb',
  'health',
  'reason',
  'replac',
  'yasuo',
  'fukuda',
  'begin',
  'string',
  'five',
  'prime',
  'minist',
  'none',
  'retain',
  'offic',
  'sixteen',
  'month',
  'stage',
  'polit',
  'comeback',
  'septemb',
  'defeat',
  'former',
  'minist',
  'defens',
  'shigeru',
  'ishiba',
  'vote',
  'presidenti',
  'elect',
  'follow',
  'landslid',
  'victori',
  'general',
  'elect',
  'becam',
  'prime',
  'minist',
  'first',
  'former',
  'prime',
  'minist',
  'return',
  'offic',
  'sinc',
  'shigeru',
  'yoshi

In [96]:
dictionary = corpora.Dictionary(processed_docs)

In [97]:
for idx, (k, v) in enumerate(dictionary.iteritems()):
    print(k, v)
    if idx >= 10:
        break

0 also
1 becam
2 begin
3 born
4 coalit
5 comeback
6 current
7 decemb
8 defeat
9 defens
10 democrat


### Model Hyperparameter tuning

In [109]:
## Model hyper parameters:

## These are the dictionary preparation parameters:
filter_tokens_if_container_documents_are_less_than = 1
filter_tokens_if_appeared_percentage_more_than = 0.5
keep_the_first_n_tokens=100000

## and the LDA Parameters: 
num_of_topics = 10

In [110]:
dictionary.filter_extremes(
        no_below=filter_tokens_if_container_documents_are_less_than, 
        no_above=filter_tokens_if_appeared_percentage_more_than, 
        keep_n=keep_the_first_n_tokens)

In [114]:
dictionary.keys()==[]

True

In [123]:
for i in range(1,50):

    filter_tokens_if_container_documents_are_less_than = 10
    filter_tokens_if_appeared_percentage_more_than = 0.5
    keep_the_first_n_tokens=10*i


    dictionary.filter_extremes(
        no_below=filter_tokens_if_container_documents_are_less_than, 
        no_above=filter_tokens_if_appeared_percentage_more_than, 
        keep_n=keep_the_first_n_tokens)
    
    if dictionary.keys()!=[]:

        print(i)

In [124]:
dictionary.keys()

[]

In [101]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

bow_corpus

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]

In [102]:
# randomly choose an article from the corpus:
sample_bow_doc = choice(bow_corpus)

print('The processed bag-of-word document is just pairs of (word_id, # of occurnces) and looks like this:')
print(sample_bow_doc, '\n\n')

print ('We peek in the dictionary: for each word_id, we get its assigned word:')
for word_id, word_freq in sample_bow_doc:
  real_word = dictionary[word_id]
  print(f'Word {word_id} ("{real_word}") appears {word_freq} time.')

The processed bag-of-word document is just pairs of (word_id, # of occurnces) and looks like this:
[] 


We peek in the dictionary: for each word_id, we get its assigned word:


## LDA model using Bag-of-words

In [103]:
lda_model = models.LdaMulticore(bow_corpus, 
                                num_topics=num_of_topics, 
                                id2word=dictionary, 
                                passes=5, 
                                workers=2)

ValueError: ignored

In [ ]:
for idx, topic in lda_model.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Words: {topic}')

## TF / IDF

In [ ]:
# initialize a tfidf from our corpus
tfidf = models.TfidfModel(bow_corpus)

# apply it on our corpus 
tfidf_corpus = tfidf[bow_corpus]

pprint(tfidf_corpus[0][:10])

In [ ]:
# the new tfidf corpus is just our corpus - but transformed. It has the same size of documents:
assert len(bow_corpus) == len(tfidf_corpus)

Now let's apply LDA on the tfidf corpus, with the same amount of topics.

You can play with the # of passes, if the model doesn't converge properly

In [ ]:
lda_model_tfidf = models.LdaMulticore(tfidf_corpus, 
                                      num_topics=num_of_topics, 
                                      id2word=dictionary, 
                                      passes=5, 
                                      workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Word: {topic}')

## Inference

Now that we have a topic-modeler, let's use it on one of the articles.

In [ ]:
# randomly pick an article:
test_doc = choice(range(len(processed_docs)))
processed_docs[test_doc][:50]

Using the original BOW model:

In [ ]:
for index, score in sorted(lda_model[bow_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print(f"Topic match score: {score} \nTopic: {lda_model.print_topic(index, num_of_topics)}")


And with the TF/IDF model:

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print("Topic match score: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, num_of_topics)))

In [ ]:
print('Perplexity: ', lda_model.log_perplexity(bow_corpus)) 
print('Perplexity TFIDF: ', lda_model_tfidf.log_perplexity(bow_corpus)) 

### Exercise - inference

Now try it on a new document!

Go to a news website, such as [orf.at](https://orf.at/) and copy an article of your choice here:

In [ ]:
unseen_document = """Sobotka sprach in einer um eineinhalb Stunden verspäteten Pressekonferenz am Abend von einer „schwierigen“, aber „sachlichen und pointierten“ Diskussion im Hauptausschuss. Er meinte, dass von Gesundheitsminister Rudolf Anschober (Grüne), die „Notwendigkeit, Verhältnismäßigkeit und Richtigkeit“ der Maßnahmen erläutert worden seien. Zudem betonte Sobotka die Wichtigkeit der im Covid-Maßnahmengesetz vorgesehenen Mitsprache des Parlaments bei Maßnahmen, bei denen auch die Freiheitsrechte betroffen seien.
Der Nationalratspräsident zitierte auch die deutsche Kanzlerin Angela Merkel, die die Maßnahmen im Kampf gegen die Pandemie als „demokratische Zumutung“ bezeichnet hatte. Dem sei nicht nur zuzustimmen, das sei auch zu unterstreichen, sagte er. Die Eindämmung der Pandemie könne aber nur gelingen, wenn sich alle Österreicher einbringen. Der Beschluss alleine reiche nicht, die Maßnahmen müssten auch im Geiste verinnerlicht werden, appellierte Sobotka an die Bevölkerung, die Maßnahmen mitzutragen.
Rendi-Wagner kritisiert Regierung
Die Opposition hatte schon vor der Sitzung – und dann auch noch einmal danach – ihre Kritik an der Regierung bekräftigt. „Die Bundesregierung hat die Kontrolle über das Infektionsgeschehen verloren und hat die schwierige Situation, vor der wir jetzt stehen, zu verantworten“, kritisierte SPÖ-Klub- und Parteivorsitzende Rendi-Wagner.
Sessel auf Tischen in einem geschlossenen Lokal
DEBATTEWie gerechtfertigt sind die Maßnahmen?"""

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

print("Simply printing the lda_model output would look like this:")
pprint(lda_model[bow_vector])

print("\n\nSo let's make it nicer, by printing the topic contents:")
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))


## Visualization

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

bow_lda_data = gensimvis.prepare(lda_model, bow_corpus, dictionary)

pyLDAvis.display(bow_lda_data)